In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("./../data/uniprot_sprot.40.multi-step.clusters/hmmsearch/A0A5P9NYS6.out", delimiter=r"\s+", comment="#", usecols=[0,2,4,5,6,7,8,9]) 
df.columns = ["sequence", "reference", "e-value", "score", "bias", "d-e-value", "d-score", "d-bias"]

In [6]:
df

,sequence,reference,e-value,score,bias,d-e-value,d-score,d-bias
0,sp|A0A193H397|CZS12_CRUCA,A0A5P9NYS6,2.700000e-42,135.1,3.8,3.000000e-42,134.9,3.8
1,sp|P81486|DRS4_PHYBI,A0A5P9NYS6,3.200000e-42,134.8,3.4,3.500000e-42,134.7,3.4
2,sp|P84597|DRS1_PITHY,A0A5P9NYS6,3.500000e-42,134.7,3.5,3.800000e-42,134.6,3.5
3,sp|A0A193H329|CZS8_CRUCA,A0A5P9NYS6,5.500000e-42,134.1,3.6,6.000000e-42,134.0,3.6
4,sp|A0A5P9K470|DRS2_AGASP,A0A5P9NYS6,5.500000e-42,134.1,4.7,6.000000e-42,133.9,4.7
...,...,...,...,...,...,...,...,...
68,sp|P69019|AUR25_RANAE,A0A5P9NYS6,4.200000e-31,99.2,2.7,4.800000e-31,99.0,2.7
69,sp|Q571V4|FRE31_NYCIN,A0A5P9NYS6,1.000000e-30,98.0,3.2,1.000000e-30,98.0,3.2
70,sp|P82390|AUR23_RANAE,A0A5P9NYS6,1.500000e-30,97.4,1.6,1.800000e-30,97.2,1.6
71,sp|L0L3V3|FRE21_SPHLA,A0A5P9NYS6,8.200000e-26,82.3,10.5,4.100000e-25,80.0,10.5
